In [40]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen

### API 요청을 통한 데이터 수집
- 국토교통부_건축물연령정보서비스
- https://www.data.go.kr/data/15057197/openapi.do

In [75]:
# 요청 url
url = 'http://apis.data.go.kr/1611000/nsdi/BuildingAgeService/attr/getBuildingAge'

# 인증키
key = 'vUmQqqkTEt8z%2Bid6c2ti6kImrxBR3ZdALI0kdi21qKxJAtgNhyJ6JaIAFeYYCiLmWzkNkyOGcYzLJWE%2FOd9CHw%3D%3D'

#필수 요청 params : pnu (고유번호(8자리 이상))

#### 1) 요청변수 불러오기
- 필수 요청 변수인 고유번호(법정동코드)를 '행정동법정동코드.xlsx'에서 추출
- 부산 지역의 법정동코드만 사용

In [85]:
code = pd.read_excel('Documents/행정동법정동코드.xlsx')

In [88]:
#부산 지역 데이터만 추출
bs_code = code[code['시도명'] == '부산광역시']

In [91]:
#필요없는 컬럼, 결측치 처리
bs_code.drop('말소일자', axis=1, inplace=True)
bs_code.dropna(inplace=True)
bs_code.reset_index(inplace=True, drop=True)

/var/folders/p9/8hvxfvwd1cbfjjyrzwqff47m0000gn/T/ipykernel_53901/4067004193.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bs_code.drop('말소일자', axis=1, inplace=True)


In [98]:
#요청변수로 사용하기 위해 [법정동코드] 컬럼의 데이터를 리스트로 저장
code_list = list(bs_code.법정동코드)

In [133]:
len(code_list)

361

#### 2) API를 활용한 데이터 수집

In [147]:
def Getdata(pnu) :
    real_url = url + '?' + 'serviceKey=' + key + '&pnu=' + pnu + '&format=' + 'xml'
    html = requests.get(real_url).text
    return html

for i in range(len(code_list)) :
    pnu = str(code_list[i])
    soup = BeautifulSoup(Getdata(pnu), 'xml')

In [148]:
soup

<?xml version="1.0" encoding="utf-8"?>
<response><numOfRows>10</numOfRows><pageNo>1</pageNo><totalCount>202</totalCount><fields><field><gisIdntfCno>1989394345522051156000000000</gisIdntfCno><pnu>2671033031100240001</pnu><ldCode>2671033031</ldCode><ldCodeNm>부산광역시 기장군 철마면 임기리</ldCodeNm><regstrSeCode>1</regstrSeCode><regstrSeCodeNm>일반</regstrSeCodeNm><mnnmSlno>24-1</mnnmSlno><buldIdntfcNo>14586</buldIdntfcNo><agbldgSeCode>1</agbldgSeCode><agbldgSeCodeNm>일반건축물</agbldgSeCodeNm><buldKndCode>2</buldKndCode><buldKndCodeNm>일반건축물대방</buldKndCodeNm><buldTotar>98.76</buldTotar><strctCode>12</strctCode><strctCodeNm>블록구조</strctCodeNm><mainPrposCode>01000</mainPrposCode><mainPrposCodeNm>단독주택</mainPrposCodeNm><buldHg>0</buldHg><groundFloorCo>1</groundFloorCo><undgrndFloorCo>0</undgrndFloorCo><useConfmDe>1989-06-27</useConfmDe><buldAge>35</buldAge><agrdeSeCode>04</agrdeSeCode><agrdeSeCodeNm>30대</agrdeSeCodeNm><agrde5ClassSeCode>040</agrde5ClassSeCode><agrde5ClassSeCodeNm>40세미만</agrde5ClassSeCodeNm><last

#### 3) 원하는 정보 추출

In [149]:
#<field> 태그 하위 항목들 가져오기
items = soup.find_all('field')

In [150]:
#얻고 싶은 데이터를 리스트에 추가 - 법정동코드 / 법정동명 / 건물연령

loc_code = []
loc = []
building_age = []

for item in items :
    try : 
        loc_code.append(item.find('ldCode').get_text())
        loc.append(item.find('ldCodeNm').get_text())
        building_age.append(item.find('buldAge').get_text())
    except : 
        pass

In [151]:
loc

['부산광역시 기장군 철마면 임기리',
 '부산광역시 기장군 철마면 임기리',
 '부산광역시 기장군 철마면 임기리',
 '부산광역시 기장군 철마면 임기리',
 '부산광역시 기장군 철마면 임기리',
 '부산광역시 기장군 철마면 임기리',
 '부산광역시 기장군 철마면 임기리',
 '부산광역시 기장군 철마면 임기리',
 '부산광역시 기장군 철마면 임기리',
 '부산광역시 기장군 철마면 임기리']

In [152]:
building_age

['35', '20', '27', '83', '10', '10', '27', '7']

In [ ]:
# 데이터 프레임으로 만들기

In [153]:
total = {'법정동코드' : loc_code, 
                  '법정동명' : loc, 
                  '건물연령' : building_age}

df = pd.DataFrame.from_dict(total, orient='index')

In [154]:
df = df.transpose()

In [155]:
df.head()

,법정동코드,법정동명,건물연령
0,2671033031,부산광역시 기장군 철마면 임기리,35
1,2671033031,부산광역시 기장군 철마면 임기리,20
2,2671033031,부산광역시 기장군 철마면 임기리,27
3,2671033031,부산광역시 기장군 철마면 임기리,83
4,2671033031,부산광역시 기장군 철마면 임기리,10


In [156]:
# pnu = '2611010200'

# real_url = url + '?' + 'serviceKey=' + key + '&pnu=' + 'pnu' + '&format=' + 'xml'

# html = requests.get(real_url).text

# soup = BeautifulSoup(html, 'xml')